## imports

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import folium
import geopy.geocoders
from geopy.geocoders import Nominatim
import geopy.distance
import codecs
import numpy as np

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

Overview: Find the average attendance for all major league teams in North America. Then model to attendance and find what features influence a teams attendance.

I scraped stats about teams for the five major league sports(MLB, NBA, NFL, NHL, and MLS). I used wikipedia and espn.com.

# MLB - Major League Baseball

In [3]:
MLB = pd.read_html('https://en.wikipedia.org/wiki/Major_League_Baseball')[2]

In [4]:
#Drop National League in the middle of table
MLB = MLB.drop([15],axis=0)

In [ ]:
#Alphabatize by team name
MLB = MLB.sort_values('Team').reset_index().drop('index',axis=1)

In [301]:
#Drop the Amercan League index in the column
def fix_wiki_columns(league_df):
    '''
    This removes the confernce that gets stuck in the table from
    a Wikipedia tables.
    '''
    if len(league_df.columns[0]) == 2:
        new_col = []
        for col in league_df.columns:
            new_col.append(col[0])
        league_df.columns = new_col
        return 
    else:
        return 

In [ ]:
fix_wiki_columns(MLB)

In [81]:
MLB.columns

Index(['Division', 'Team', 'City', 'Stadium', 'Capacity', 'Coordinates',
       'Founded', 'Joined', 'Ref'],
      dtype='object')

In [82]:
MLB = MLB.drop(columns=['Division','Ref'])

In [316]:
#Clean the Coordinates
def fix_wiki_coordinates(league_df):
    '''
    Reformat the Arena coordinates from the wikipedia table.
    '''
    new_coords, lat, long = [], [], []
    
    for row in league_df.Coordinates.str.split('/'):
        coord = row[1].split('\ufeff')[1].split()
        
        coord_lat = float(coord[0].strip('°N'))
        coord_long = -1*float(coord[1].strip('°W'))

        new_coords.append((coord_lat, coord_long))
        lat.append(coord_lat)
        long.append(coord_long)

    league_df['Coordinates'] = new_coords
    league_df['Latitude'] = lat
    league_df['Longitude'] = long

In [ ]:
fix_wiki_coordinates(MLB)

In [346]:
def fix_wiki_year(league_df):
    league_df['Founded'] = league_df.Founded.str.strip('*()NLA ').astype(int)
    league_df['Joined'] = league_df.Joined.str.strip('*()NLA ').astype(int)

In [347]:
fix_wiki_year(MLB)

## Add attendance

I needed to find attendance on another website. This is the case for all the sports.

In [202]:
mlb_attend = pd.read_html('http://www.espn.com/mlb/attendance')

In [203]:
mlb_attend = mlb_attend[0].drop([0,3,5,6,7,8,9,10,11], axis=1)

In [204]:
mlb_attend.columns = mlb_attend.loc[1]

In [205]:
mlb_attend = mlb_attend.drop([0,1])

In [260]:
mlb_attend.columns = ['Team','Games','AvgAttendance']

In [241]:
mlb_attend = mlb_attend.sort_values('TEAM').reset_index().drop('index',axis=1)

In [269]:
MLB = MLB.join(mlb_attend['AvgAttendance'])
MLB = MLB.join(mlb_attend['Games'])

In [226]:
MLB['Games'] = mlb_attend.sort_values('TEAM').GMS

In [232]:
MLB = MLB.drop(['AvgAttendance', 'Games'],axis=1)

In [253]:
MLB = MLB.drop(['level_0'],axis=1)

In [349]:
MLB.to_csv('MLB.csv')

# NBA

In [319]:
NBA = pd.read_html('https://en.wikipedia.org/wiki/National_Basketball_Association')

In [320]:
#select specific table
NBA = NBA[2]

In [321]:
#Drop conference row that was a divider
NBA = NBA.drop([15],axis=0)

In [322]:
fix_wiki_columns(NBA)

In [323]:
#Alphabatize by team name
NBA = NBA.sort_values('Team').reset_index().drop('index',axis=1)

In [325]:
NBA = NBA.drop(columns=['Division','Unnamed: 8_level_0'])

In [326]:
fix_wiki_coordinates(NBA)

In [350]:
fix_wiki_year(NBA)

## Add Indices

In [419]:
team_index = []
for team in NBA.Team:
    team_index.append(team.split()[-1].lower())

NBA['team_index'] = team_index

NBA = NBA.set_index('team_index')

## Add attendance

In [367]:
nba_attend = pd.read_html('http://www.espn.com/nba/attendance/_/year/2019')

In [370]:
nba_attend = nba_attend[0].drop([0,3,5,6,7,8,9,10,11], axis=1)
nba_attend = nba_attend.drop([32,33,34,35], axis=0)

In [372]:
nba_attend.columns = nba_attend.loc[1]
nba_attend = nba_attend.drop([0,1])
nba_attend.columns = ['Team','Games','AvgAttendance']

In [374]:
nba_attend = nba_attend.sort_values('Team').reset_index().drop('index',axis=1)

### add indices to attendance table

In [420]:
team_index = []
for team in nba_attend.Team:
    if len(team.split()) == 2:
        team_index.append(team.split()[-1].lower())
    else:
        team_index.append(team.lower())
    #print(team)

nba_attend['team_index'] = team_index

nba_attend = nba_attend.set_index('team_index')

### Add new attendance columns

In [429]:
NBA = NBA.join(nba_attend.AvgAttendance)
NBA = NBA.join(nba_attend.Games)

In [431]:
NBA.to_csv('NBA.csv')

# NFL

In [432]:
NFL = pd.read_html('https://en.wikipedia.org/wiki/National_Football_League')

In [435]:
#select specific table
NFL = NFL[2]

## Cleaning NFL table

In [436]:
#Drop conference row that was a divider
NFL = NFL.drop([16,33],axis=0)

In [438]:
#Remove conference in the header
fix_wiki_columns(NFL)

In [440]:
NFL = NFL.drop(columns=['Division[57]','Head coach'])

In [442]:
NFL.columns = ['Team','City','Stadium','Capacity','Coordinates','First_Season']

In [444]:
#Alphabatize by team name
NFL = NFL.sort_values('Team').reset_index().drop('index',axis=1)

In [447]:
fix_wiki_coordinates(NFL)

In [1108]:
NFL['Founded'] = NFL.First_Season.str[0:4].astype(int)

In [1113]:
NFL = NFL.drop(['First_Season'],axis=1)

In [451]:
NFL['Team'] = NFL.Team.str.strip('*†')

In [465]:
#index is team
team_index = []
for team in NFL.Team:
    if len(team.split()) == 2:
        team_index.append(team.split()[0].lower())
    elif team.find('New York') != -1:
        team_sym = team.split()[-1].lower()
        team_index.append('ny '+team_sym)
    elif team.find('Los Angeles') != -1:
        team_sym = team.split()[-1].lower()
        team_index.append('la '+team_sym)
    else:
        city0 = team.split()[0].lower()
        city1 = team.split()[1].lower()
        team_index.append(city0+' '+city1)
        
NFL['team_index'] = team_index
NFL = NFL.set_index('team_index')

## Add attendance

In [493]:
nfl_attend = pd.read_html('http://www.espn.com/nfl/attendance/_/year/2019')

In [494]:
nfl_attend = nfl_attend[0].drop([0,3,5,6,7,8,9,10,11,12,13], axis=1)

In [495]:
nfl_attend = nfl_attend.drop([0,1])
nfl_attend.columns = ['Team','Games','AvgAttendance']

### add indices

In [496]:
nfl_attend.loc[12,'Team'] = 'la rams'
nfl_attend.loc[33,'Team'] = 'la chargers'

In [500]:
nfl_attend['Team'] = nfl_attend.Team.str.lower()

In [501]:
nfl_attend = nfl_attend.set_index('Team')

### Add new attendance columns

In [504]:
NFL = NFL.join(nfl_attend.AvgAttendance)
NFL = NFL.join(nfl_attend.Games)

In [505]:
NFL.to_csv('NFL.csv')

# NHL

In [506]:
NHL = pd.read_html('https://en.wikipedia.org/wiki/National_Hockey_League')

In [509]:
#select specific table
NHL = NHL[2]

In [511]:
#Drop conference row that was a divider
NHL = NHL.drop([16],axis=0)

In [512]:
fix_wiki_columns(NHL)

In [515]:
NHL = NHL.drop(columns=['Division','General manager','Head coach','Captain'])

In [517]:
fix_wiki_year(NHL)

## Add indices to the NHL table

In [522]:
#index based on team
team_index = []
for team in NHL.Team:
    if team.find('New York') != -1:
        team_sym = team.split()[-1].lower()
        team_index.append('ny '+team_sym)
    
    else:
        team_index.append(team.split()[0].lower())
        
NHL['team_index'] = team_index
NHL = NHL.set_index('team_index')

## Add attendance

In [523]:
nhl_attend = pd.read_html('http://www.espn.com/nhl/attendance/_/year/2019')

In [526]:
nhl_attend = nhl_attend[0].drop([0,3,5,6,7,8,9,10,11], axis=1)

In [529]:
nhl_attend = nhl_attend.drop([0,1])
nhl_attend.columns = ['Team','Games','AvgAttendance']

### add indices

In [534]:
#index based on team
team_index = []
for team in nhl_attend.Team:
    if team.find('NY') != -1:
        team_sym = team.split()[-1].lower()
        team_index.append('ny '+team_sym)
    
    else:
        team_index.append(team.split()[0].lower())
        
nhl_attend['team_index'] = team_index
nhl_attend = nhl_attend.set_index('team_index')

### Add new attendance columns

In [537]:
NHL = NHL.join(nhl_attend.AvgAttendance)
NHL = NHL.join(nhl_attend.Games)

In [545]:
NHL.loc['ny islanders','City'] ='New York City, New York'
NHL.loc['ny islanders','Arena'] = 'Barclays Center'
NHL.loc['ny islanders','Capacity'] = 15795
NHL.loc['arizona','Team'] = 'Arizona Coyotes'

### Add coordinates

The NHL team details didn't include coordinates for the stadium, so I needed to get them. I used geopy to get them.

In [552]:
geopy.geocoders.options.default_user_agent = 'Metis Project 2'
geopy.geocoders.options.default_timeout = 15
geolocator = Nominatim()

In [619]:
def arena_coordinates(league_df,arena='Arena',city='City'):
    '''
    Findes the coordinates of the arenas in the league. Default column names are Arena and City
    '''
    lats, longs, coords = [], [], []
    
    for stadium in league_df[arena]+' '+league_df[city]:
        location = geolocator.geocode(stadium)
        time.sleep(5)
        
        try:
            lats.append(location.latitude)
            longs.append(location.longitude)
            coords.append((location.latitude,location.longitude))
        except:
            lats.append(np.NaN)
            longs.append(np.NaN)
            coords.append(np.NaN)
            print("Couldn't find ",stadium)
    
    league_df['Coordinates'] = coords
    league_df['Latitude'] = lats
    league_df['Longitude'] = longs
    return

In [620]:
arena_coordinates(NHL)

Couldn't find  SAP Center San Jose, California


In [637]:
coord = (37.3327507,-121.9012447)
NHL.loc['san','Coordinates'] = coord 

ValueError: Must have equal len keys and value when setting with an iterable

In [629]:
NHL.loc['san','Latitude'] = 37.3327507
NHL.loc['san','Longitude'] = -121.9012447

In [826]:
NHL.loc['san']

Team                  San Jose Sharks
City             San Jose, California
Arena                      SAP Center
Capacity                        17562
Founded                          1991
Joined                           1991
AvgAttendance                   17266
Games                              41
Coordinates                       NaN
Latitude                      37.3328
Longitude                    -121.901
Name: san, dtype: object

In [622]:
NHL.to_csv('NHL.csv')

# MLS

In [848]:
MLS = pd.read_html('https://en.wikipedia.org/wiki/Major_League_Soccer')

In [849]:
MLS = MLS[1]

In [850]:
#Drop conference row that was a divider
MLS = MLS.drop([13],axis=0)

In [851]:
fix_wiki_columns(MLS)

In [852]:
MLS = MLS.drop(columns=['Head coach'])

In [918]:
MLS.Team[(MLS.Joined == '2020')]

Team
Inter Miami CF    Inter Miami CF
Nashville SC        Nashville SC
Name: Team, dtype: object

In [921]:
MLS = MLS.drop(['Inter Miami CF','Nashville SC'],axis=0)

In [855]:
MLS.Arena = MLS.Arena.str.strip('123 ')

In [856]:
MLS.index = MLS.Team

In [866]:
MLS.loc['Chicago Fire FC','City, State'] = 'Bridgeview, Illinois'
MLS.loc['Chicago Fire FC','Arena'] = 'SeatGeek Stadium'

In [882]:
capacities = []
for i, cap in enumerate(MLS.Capacity):
    if len(cap) == 6:
        capacities.append(cap[0:-1])
    else:
        capacities.append(cap)
MLS.Capacity = capacities

## Add attendance

In [937]:
mls_attend = pd.read_html('https://en.wikipedia.org/wiki/Major_League_Soccer_attendance#Doubleheader_games_highest_attendance')

In [938]:
mls_attend = mls_attend[0]

In [949]:
mls_attend.Team[23] = 'Chicago Fire FC'

/Users/nkfoster/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [948]:
mls_attend.Team[20] = 'Columbus Crew SC'

/Users/nkfoster/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [951]:
mls_attend = mls_attend.set_index('Team')

In [952]:
mls_attend.columns = ['AvgAttendance',1,2,3,4,5]

In [955]:
MLS = MLS.join(mls_attend['AvgAttendance'])

In [1196]:
MLS['Games'] = 17

## Add coordinates

In [957]:
arena_coordinates(MLS, city='City, State')

Couldn't find  Dignity Health Sports Park Carson, California


In [965]:
MLS.loc['LA Galaxy','Latitude'] = 33.8643643 
MLS.loc['LA Galaxy','Longitude'] = -118.261207

In [967]:
MLS.to_csv('MLS.csv')

# Add standings

Team win records had to be added from another website. I standardized the win percentages. Each win was worth a point and a tie was worth .5. Then the total was divided by the number of games.

In [732]:
def get_ESPN_standings(url):
    standings = pd.read_html(url)
    #fix team columns
    first_row = pd.DataFrame({'Team' : [standings[0].columns[0]]})
    standings[0].columns = ['Team']
    team_standings = first_row.append(standings[0]).reset_index()
    team_standings = team_standings.drop('index', axis=1)

    team_standings = team_standings.join(standings[1])
    
    team_index = []
    for team in team_standings.Team:
        if team.find('Sox') == -1:
            team_index.append(team.split()[-1].lower())
        else:
            team_index.append(team.split()[-2].lower()+' '+team.split()[-1].lower())
    team_standings['team_index'] = team_index
    team_standings = team_standings.set_index('team_index')
    return team_standings


In [969]:
def reindex_league(league_df):
    team_index = []
    for team in league_df.Team:
        if team.find('Sox') == -1:
            team_index.append(team.split()[-1].lower())
        else:
            team_index.append(team.split()[-2].lower()+' '+team.split()[-1].lower())
    league_df.index = team_index
    #league_df = league_df.set_index('team_index')

## MLB

In [733]:
mlb_standings = get_ESPN_standings('https://www.espn.com/mlb/standings/_/season/2018/group/overall')

In [734]:
mlb_standings

,Team,W,L,PCT,GB,HOME,AWAY,RS,RA,DIFF,STRK,L10
team_index,,,,,,,,,,,,
red sox,* --BOSBoston Red Sox,108,54,0.667,-,57-24,51-30,876,647,229,W1,5-5
astros,x --HOUHouston Astros,103,59,0.636,5,46-35,57-24,797,534,263,L1,8-2
yankees,y --NYYNew York Yankees,100,62,0.617,8,53-28,47-34,851,669,182,L1,7-3
athletics,y --OAKOakland Athletics,97,65,0.599,11,50-31,47-34,813,674,139,L1,6-4
brewers,* --MILMilwaukee Brewers,96,67,0.589,12.5,51-30,45-37,754,659,95,W8,9-1
cubs,y --CHCChicago Cubs,95,68,0.583,13.5,51-31,44-37,761,645,116,L1,6-4
dodgers,x --LADLos Angeles Dodgers,92,71,0.564,16.5,45-37,47-34,804,610,194,W4,7-3
indians,x --CLECleveland Indians,91,71,0.562,17,49-32,42-39,818,648,170,W1,6-4
rockies,y --COLColorado Rockies,91,72,0.558,17.5,47-34,44-38,780,745,35,L1,8-2


In [970]:
reindex_league(MLB)

In [973]:
MLB = MLB.join(mlb_standings.PCT)

## NBA

In [735]:
nba_standings = get_ESPN_standings('https://www.espn.com/nba/standings/_/season/2018/group/league')

In [736]:
nba_standings

,Team,W,L,PCT,GB,HOME,AWAY,DIV,CONF,PPG,OPP PPG,DIFF,STRK,L10
team_index,,,,,,,,,,,,,,
rockets,* --HOUHouston Rockets,65,17,0.793,-,34-7,31-10,12-4,41-11,112.4,103.9,8.5,L1,7-3
raptors,z --TORToronto Raptors,59,23,0.720,6,34-7,25-16,12-4,40-12,111.7,103.9,7.8,L1,6-4
warriors,y --GSGolden State Warriors,58,24,0.707,7,29-12,29-12,13-3,34-18,113.5,107.5,6.0,L1,4-6
celtics,x --BOSBoston Celtics,55,27,0.671,10,27-14,28-13,12-4,33-19,104.0,100.4,3.6,W1,6-4
76ers,x --PHIPhiladelphia 76ers,52,30,0.634,13,30-11,22-19,9-7,34-18,109.8,105.3,4.5,W16,10-0
cavaliers,y --CLECleveland Cavaliers,50,32,0.610,15,29-12,21-20,11-5,35-17,110.9,109.9,1.0,L1,7-3
blazers,y --PORPortland Trail Blazers,49,33,0.598,16,28-13,21-20,9-7,31-21,105.6,103.0,2.6,W1,5-5
thunder,x --OKCOklahoma City Thunder,48,34,0.585,17,27-14,21-20,5-11,28-24,107.9,104.4,3.5,W3,5-5
jazz,x --UTAHUtah Jazz,48,34,0.585,17,28-13,20-21,7-9,34-18,104.1,99.8,4.3,L1,7-3


In [976]:
NBA = NBA.join(nba_standings.PCT)

## NFL

In [737]:
nfl_standings = get_ESPN_standings('https://www.espn.com/nfl/standings/_/season/2018/group/league')

In [738]:
nfl_standings

,Team,W,L,T,PCT,HOME,AWAY,DIV,CONF,PF,PA,DIFF,STRK
team_index,,,,,,,,,,,,,
saints,** --NONew Orleans Saints,13,3,0,0.813,6-2,7-1,4-2,9-3,504,353,151,L1
rams,* --LARLos Angeles Rams,13,3,0,0.813,7-1,6-2,6-0,9-3,527,384,143,W2
chiefs,** --KCKansas City Chiefs,12,4,0,0.750,7-1,5-3,5-1,10-2,565,421,144,W1
bears,z --CHIChicago Bears,12,4,0,0.750,7-1,5-3,5-1,10-2,421,283,138,W4
chargers,y --LACLos Angeles Chargers,12,4,0,0.750,5-3,7-1,4-2,9-3,428,329,99,W1
patriots,* --NENew England Patriots,11,5,0,0.688,8-0,3-5,5-1,8-4,436,325,111,W2
texans,z --HOUHouston Texans,11,5,0,0.688,6-2,5-3,4-2,9-3,402,316,86,W1
ravens,z --BALBaltimore Ravens,10,6,0,0.625,6-2,4-4,3-3,8-4,389,287,102,W3
cowboys,z --DALDallas Cowboys,10,6,0,0.625,7-1,3-5,5-1,9-3,339,324,15,W2


In [978]:
reindex_league(NFL)

In [981]:
NFL = NFL.join(nfl_standings.PCT)

## NHL

In [739]:
nhl_standings = get_ESPN_standings('https://www.espn.com/nhl/standings/_/sort/points/dir/desc/season/2018/group/league')

In [984]:
nhl_standings['PCT'] = (nhl_standings.PTS.astype(float)/82/2).round(3)

In [987]:
reindex_league(NHL)

In [994]:
NHL = NHL.join(nhl_standings.PCT)

## MLS

In [1027]:
mls_standings = pd.read_html('https://www.espn.com/soccer/standings/_/league/USA.1/season/2018')

In [1028]:
mls_standings[0].columns = ['Team']

In [1029]:
mls_standings = mls_standings[0].join(mls_standings[1])

In [1031]:
mls_standings = mls_standings.drop([0,12],axis=0)
mls_standings = mls_standings.drop([4,5,6,7],axis=1)
mls_standings.columns = ['Team','GP','W','D','L']

In [1032]:
#Reindex
trim = [3,4,4,3,4,4,4,3,4,5,5,4,4,5,4,4,4,3,4,4,5,5,4]
team_index = []
for i, team in enumerate(mls_standings.Team):
    team_index.append(team[trim[i]:])
mls_standings.index = team_index

In [1037]:
mls_standings['PCT'] = ((mls_standings.W.astype(float)*2 + mls_standings.D.astype(float))/34/2).round(3)

In [1038]:
MLS = MLS.sort_index()

In [1039]:
mls_standings = mls_standings.sort_index()

In [ ]:
MLS = MLS.drop(['FC Cincinnati'])

In [1043]:
MLS.index = mls_standings.index

In [1046]:
MLS = MLS.join(mls_standings.PCT)

# Add distance

In [1130]:
def coord_as_floats(league_df):
    league_df.Latitude = league_df.Latitude.astype(float)
    league_df.Longitude = league_df.Longitude.astype(float)
    league_df = league_df.drop(columns='Coordinates')

In [1063]:
def dist_from_other_stadiums(loc, locations):
    distances = []
    for location in locations:
        if location != loc:
            dist = geopy.distance.distance(loc, location).mi
            distances.append(dist)
    return distances

In [1096]:
def dist_closest_team(league_df):
    distances = []
    for i, team in enumerate(league_df.Team):
        min_dist = min(
            dist_from_other_stadiums(
                (league_df.Latitude[i], league_df.Longitude[i]),
                zip(league_df.Latitude, league_df.Longitude)))
        distances.append(round(min_dist,1))
    league_df['Dist_close_team_in_league'] = distances

In [1237]:
def number_of_teams_in_range(league_df, range_=10):
    teams_in_range = []
    for i, team in enumerate(league_df.Team):
        distances = dist_from_other_stadiums(
            (league_df.Latitude[i], league_df.Longitude[i]),
            zip(league_df.Latitude, league_df.Longitude))
        count = 0
        for distance in distances:
            if distance <= range_:
                count += 1
        teams_in_range.append(count)
    league_df['Teams_within_' + str(range_) + '_mi'] = teams_in_range

In [1139]:
dist_closest_team(MLB)
dist_closest_team(NBA)
dist_closest_team(NFL)
dist_closest_team(NHL)
dist_closest_team(MLS)

# Combine leagues

In [1143]:
MLB['Sport'] = 'Baseball'
NBA['Sport'] = 'Basketball'
NFL['Sport'] = 'Football'
NHL['Sport'] = 'Hockey'
MLS['Sport'] = 'Soccer'

In [1158]:
MLS['Founded'] = MLS.Joined

In [1170]:
NBA['City'] = NBA['City, State']
MLS['City'] = MLS['City, State']

In [1202]:
def add_venue(league_df):
    try:
        league_df['Venue'] = league_df.Stadium
    except:
        league_df['Venue'] = league_df.Arena

In [1206]:
add_venue(MLB)
add_venue(NBA)
add_venue(NFL)
add_venue(NHL)
add_venue(MLS)

In [1475]:
leagues = pd.concat([MLB,NBA,NFL,NHL,MLS],sort=False)

In [1476]:
leagues = leagues.drop(columns=['Stadium','Coordinates','Joined','City, State','Arena'])

In [1477]:
number_of_teams_in_range(leagues, range_=1)

In [1478]:
number_of_teams_in_range(leagues, range_=10)

In [1479]:
number_of_teams_in_range(leagues, range_=25)

In [1480]:
number_of_teams_in_range(leagues, range_=50)

# Population

I added population data for the cities of the stadiums. I had to find data for both the US and Canada. Some stadiums were located in suburbs that were not represented on the table. I set those to 50,000. That was the cut off in population for the cities listed.

In [1382]:
canpop = pd.read_html('https://en.wikipedia.org/wiki/List_of_the_100_largest_municipalities_in_Canada_by_population')

In [1383]:
canpop = canpop[0]

In [1384]:
canpop['City'] = canpop.Municipality+', '+canpop.Province

In [1385]:
canpop.columns = [0,1,2,3,'Land_Area_km2',5,'pop2016',7,8,9,10,'City']

In [1386]:
canpop['land_area'] = (canpop.Land_Area_km2/2.59).astype(int)

In [1387]:
canpop['pop_dens'] = (canpop.pop2016 / canpop.land_area).astype(int)

In [1388]:
canpop = canpop[['City','pop_dens','pop2016','land_area']]

In [1389]:
canpop.index = canpop.City

In [1425]:
uspop = pd.read_html('https://www.governing.com/gov-data/population-density-land-area-cities-map.html')

In [1426]:
uspop = uspop[0]

In [1427]:
uspop.index = uspop.City

In [1428]:
uspop.columns = ['City','pop_dens','pop2016','land_area']

In [1429]:
uspop.loc['New York, New York', 'City'] = 'New York City, New York'

In [1430]:
uspop.loc['St. Paul, Minnesota', 'City'] = 'Saint Paul, Minnesota'

In [1431]:
uspop.loc['Washington, District of Columbia','City'] = 'Washington, D.C.'

In [1432]:
uspop.index = uspop.City

In [1433]:
pop = pd.concat([uspop,canpop], sort=False)

In [1481]:
leagues.index = leagues.City

In [1482]:
leagues = leagues.join(pop.pop2016)

In [1500]:
leagues.isna().value_counts()

False    146
Name: Latitude, dtype: int64

In [1504]:
# Source for population data doesn't record cities less than 50,000
leagues = leagues.fillna(50000)

## Add population density from nearest city

There were some cities missing. I set them the the density of the city the team represented. I decided that it would best represent the desity of the metro areas that the teams played.

In [1449]:
leagues = leagues.join(pop.pop_dens)

In [1536]:
leagues = leagues.drop(['pop_dens'],axis=1)

In [1537]:
leagues = leagues.replace({
    'City': {
        'Bridgeview, Illinois': 'Chicago, Illinois',
        'Chester, Pennsylvania': 'Philadelphia, Pennsylvania',
        'East Rutherford, New Jersey': 'New York City, New York',
        'Foxborough, Massachusetts': 'Boston, Massachusetts',
        'Harrison, New Jersey': 'New York City, New York',
        'Landover, Maryland': 'Washington, D.C.',
        'Orchard Park, New York': 'Buffalo, New York',
        'Paradise, Nevada': 'Las Vegas, Nevada'
    }
})

In [1538]:
leagues.index = leagues.City

In [1539]:
leagues = leagues.join(pop.pop_dens)

In [1540]:
leagues[(leagues.pop_dens.isna())].head()

,Team,City,Capacity,Founded,Latitude,Longitude,AvgAttendance,Games,PCT,Dist_close_team_in_league,Sport,Venue,Teams_within_1_mi,Teams_within_10_mi,Teams_within_25_mi,Teams_within_50_mi,pop2016,pop_dens
City,,,,,,,,,,,,,,,,,,


In [1564]:
leagues.columns = [
    'Team', 'City', 'Capacity', 'Founded', 'Latitude', 'Longitude',
    'AvgAttendance', 'Games', '2018winPCT', 'Dist_close_team_in_league', 'Sport',
    'Venue', 'Teams_within_1_mi', 'Teams_within_10_mi', 'Teams_within_25_mi',
    'Teams_within_50_mi', 'pop2016', 'pop_dens'
]

In [1565]:
leagues

,Team,City,Capacity,Founded,Latitude,Longitude,AvgAttendance,Games,2018winPCT,Dist_close_team_in_league,Sport,Venue,Teams_within_1_mi,Teams_within_10_mi,Teams_within_25_mi,Teams_within_50_mi,pop2016,pop_dens
0,Los Angeles Angels,"Anaheim, California",45957,1961,33.800280,-117.882780,37321,81,0.494,27.9,Baseball,Angel Stadium,1,1,3,9,351043,7043
1,Anaheim Ducks,"Anaheim, California",17174,1993,33.807832,-117.876534,16814,41,0.616,27.7,Hockey,Honda Center,1,1,3,9,351043,7043
2,Texas Rangers,"Arlington, Texas",48114,1961,32.751390,-97.082780,26333,81,0.414,230.2,Baseball,Globe Life Field,1,1,3,4,392772,4096
3,Dallas Cowboys,"Arlington, Texas",80000,1960,32.748000,-97.093000,90929,8,0.625,233.3,Football,AT&T Stadium,1,1,3,4,392772,4096
4,Atlanta Braves,"Atlanta, Georgia",41500,1871,33.890000,-84.467780,32776,81,0.556,359.1,Baseball,Truist Park,0,0,3,3,472522,3549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,Washington Nationals,"Washington, D.C.",41313,1969,38.872780,-77.007500,27898,81,0.506,35.1,Baseball,Nationals Park,1,4,4,6,681170,11158
142,Washington Wizards,"Washington, D.C.",20356,1961,38.898056,-77.020833,17487,41,0.524,120.7,Basketball,Capital One Arena,1,4,4,6,681170,11158
143,Washington Capitals,"Washington, D.C.",18506,1974,38.898184,-77.020938,18508,41,0.640,120.7,Hockey,Capital One Arena,1,4,4,6,681170,11158
144,D.C. United,"Washington, D.C.",20000,1996,38.868215,-77.012789,17744,17,0.544,110.0,Soccer,Audi Field,1,4,4,6,681170,11158


# Save as csv

In [1552]:
leagues = leagues.reset_index(drop=True)

In [1566]:
leagues.to_csv('leagues.csv')